In [1]:
import pandas as pd
import numpy as np

df= pd.read_csv('tn.movie_budgets.csv.gz')
df

In [2]:
df= pd.read_csv('tn.movie_budgets.csv.gz') 
df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [3]:
budg= pd.read_csv('tn.movie_budgets.csv.gz')
budg['production_budget']=budg['production_budget'].str.replace(',','')
budg['domestic_gross']=budg['domestic_gross'].str.replace(',','')
budg['worldwide_gross']=budg['worldwide_gross'].str.replace(',','')
budg['production_budget']=budg['production_budget'].str.replace('$','').astype('int64')
budg['domestic_gross']=budg['domestic_gross'].str.replace('$','').astype('int64')
budg['worldwide_gross']=budg['worldwide_gross'].str.replace('$','').astype('int64')
budg.sort_values('worldwide_gross', ascending=False)
budg['net_pre_marketing']=budg['worldwide_gross']-budg['production_budget']
budg

<ipython-input-3-d7bdf45cb627>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  budg['production_budget']=budg['production_budget'].str.replace('$','').astype('int64')
<ipython-input-3-d7bdf45cb627>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  budg['domestic_gross']=budg['domestic_gross'].str.replace('$','').astype('int64')
<ipython-input-3-d7bdf45cb627>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  budg['worldwide_gross']=budg['worldwide_gross'].str.replace('$','').astype('int64')


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_pre_marketing
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000
5778,79,"Apr 2, 1999",Following,6000,48482,240495,234495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400


In [20]:
gross= pd.read_csv('bom.movie_gross.csv.gz')
gross.sort_values('domestic_gross', ascending=False)
gross['foreign_gross']=gross['foreign_gross'].str.replace(',','').astype('float64')
gross['worldwide_gross']=gross['domestic_gross']+gross['domestic_gross']
gross.sort_values('worldwide_gross', ascending=False)
gross=gross.dropna()
gross.sort_values('worldwide_gross', ascending= False)


,title,studio,domestic_gross,foreign_gross,year,worldwide_gross
1872,Star Wars: The Force Awakens,BV,936700000.0,1131.6,2015,1.873400e+09
3080,Black Panther,BV,700100000.0,646900000.0,2018,1.400200e+09
3079,Avengers: Infinity War,BV,678800000.0,1369.5,2018,1.357600e+09
1873,Jurassic World,Uni.,652300000.0,1019.4,2015,1.304600e+09
727,Marvel's The Avengers,BV,623400000.0,895500000.0,2012,1.246800e+09
...,...,...,...,...,...,...
292,Perrier's Bounty,IFC,800.0,167000.0,2010,1.600000e+03
1738,Jackpot,DR,800.0,1100000.0,2014,1.600000e+03
2920,Amityville: The Awakening,W/Dim.,700.0,7700000.0,2017,1.400000e+03
642,Illegal (2011),FM,700.0,289000.0,2011,1.400000e+03


In [ ]:
title= pd.read_csv('imdb.title.akas.csv.gz')
title

In [ ]:
name= pd.read_csv('imdb.name.basics.csv.gz')
name

In [ ]:
titles_bas= pd.read_csv('imdb.title.basics.csv.gz')
titles_bas

In [ ]:
title_crew= pd.read_csv('imdb.title.crew.csv.gz')
title_crew

In [ ]:
title_prin= pd.read_csv('imdb.title.principals.csv.gz')
title_prin

In [ ]:
title_ratings= pd.read_csv('imdb.title.ratings.csv.gz')
title_ratings

In [ ]:
rotten_rev= pd.read_csv('rotten_tomatoes_critic_reviews.csv.gz')
rotten_rev

In [ ]:
rotten_mov= pd.read_csv('rotten_tomatoes_movies.csv.gz')
rotten_mov


In [ ]:
tmdbmov= pd.read_csv('tmdb.movies.csv.gz')
tmdbmov
